In [1]:
import matplotlib.pyplot as plt
from moabb.datasets import PhysionetMI
from moabb.paradigms import MotorImagery
import numpy as np
from numpy.random import RandomState
import pandas as pd

In [2]:
def get_dataset():
  ds_src2 = PhysionetMI()
  fmin, fmax = 4, 32
  prgm_4classes = MotorImagery(n_classes=4, resample=160, fmin=fmin, fmax=fmax)
  train_data = list()
  labels = list()
  for x in range(1, 21):
      label = np.zeros(20, dtype=bool) #109 classes (persons)
      label[x-1]=1
      X_src2, label_src2, m_src2 = prgm_4classes.get_data(dataset=ds_src2, subjects=[x])
      train_data.append(X_src2)
      label_src2 = np.tile(x, (label_src2.size,1))
      # label_src2 = np.expand_dims(label_src2, axis=1)
      labels.append(label_src2)
  return train_data, labels

Choosing from all possible events


In [3]:
training_dataset, training_labels = get_dataset()

Second source dataset has 90 trials with 64 electrodes and 481 time samples


In [5]:
train_data = np.empty([1, 64, 481], dtype=np.float32)
train_data = np.vstack(training_dataset)

x = np.array(training_labels).reshape(-1,).tolist()
test_label = np.hstack(x)

Source dataset 2 include labels: (90,)


In [ ]:
# Load necessary libraries
import mne
from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

# Models
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [ ]:
#svm
clf_svm_pip = make_pipeline(Vectorizer(), StandardScaler(), svm.SVC(random_state=42))
parameters = {'svc__kernel':['linear', 'rbf', 'sigmoid'], 'svc__C':[0.1, 1, 10]}
gs_cv_svm = GridSearchCV(clf_svm_pip, parameters, scoring='accuracy', cv=StratifiedKFold(n_splits=5), return_train_score=True)

In [ ]:
gs_cv_svm.fit(train_data, test_label)
print('Best Parameters: {}'.format(gs_cv_svm.best_params_))
print('Best Score: {}'.format(gs_cv_svm.best_score_))